In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import numpy
from keras.optimizers import Adam
import keras
from matplotlib import pyplot
from keras.callbacks import EarlyStopping
import pandas as pd 
import numpy as np
from sklearn.preprocessing import LabelEncoder 

In [61]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
# Load MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Preprocess data
train_images = train_images.reshape((train_images.shape[0], 28 * 28))  # Flatten images
test_images = test_images.reshape((test_images.shape[0], 28 * 28))

# Normalize pixel values
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

# One-hot encode labels
num_classes = 10  # Number of digits (0-9)
train_labels = to_categorical(train_labels, num_classes)
test_labels = to_categorical(test_labels, num_classes)

# Define the model
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(28 * 28,)))
# Hidden layer with 512 neurons
model.add(Dense(512, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))  # Output layer with 10 neurons (one for each digit)


lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1000,
    decay_rate=0.8)


def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr # I use ._decayed_lr method instead of .lr
    return lr

optimizer = keras.optimizers.Adam(lr_schedule)
lr_metric = get_lr_metric(optimizer)


# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=optimizer , metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10, batch_size=None)

# Evaluate the model on test set
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

# Make predictions (optional)
predictions = model.predict(test_images)

Epoch 1/10
1875/1875 [==============================] - 20s 10ms/step - loss: 0.1959 - accuracy: 0.9406
Epoch 2/10
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0713 - accuracy: 0.9784
Epoch 3/10
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0365 - accuracy: 0.9890
Epoch 4/10
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0187 - accuracy: 0.9941
Epoch 5/10
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0087 - accuracy: 0.9977
Epoch 6/10
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0039 - accuracy: 0.9989
Epoch 7/10
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0017 - accuracy: 0.9995
Epoch 8/10
1875/1875 [==============================] - 21s 11ms/step - loss: 8.3214e-04 - accuracy: 0.9998
Epoch 9/10
1875/1875 [==============================] - 20s 11ms/step - loss: 4.4586e-04 - accuracy: 0.9999
Epoch 10/10
313/313 [==============================] - 1

In [59]:
import tensorflow as tf
import os

def get_mkl_enabled_flag():

    mkl_enabled = False
    major_version = int(tf.__version__.split(".")[0])
    minor_version = int(tf.__version__.split(".")[1])
    if major_version >= 2:
        if minor_version < 5:
            from tensorflow.python import _pywrap_util_port
        else:
            from tensorflow.python.util import _pywrap_util_port
            onednn_enabled = int(os.environ.get('TF_ENABLE_ONEDNN_OPTS', '0'))
        mkl_enabled = _pywrap_util_port.IsMklEnabled() or (onednn_enabled == 1)
    else:
        mkl_enabled = tf.pywrap_tensorflow.IsMklEnabled()
    return mkl_enabled

print ("We are using Tensorflow version", tf.__version__)
print("MKL enabled :", get_mkl_enabled_flag())

We are using Tensorflow version 2.15.0
MKL enabled : True
